# Linked Structures

## Agenda

1. Motivation
2. Wishlist
3. Mechanisms
4. Linked Data Structures

## 1. Motivation: built-in, array-backed list runtimes

What are the runtime complexities of some common operations for updating an array-backed list?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from timeit import timeit
%matplotlib inline

### Index-based update

In [ ]:
size = 10_000

ts = [timeit('lst[{}] = None'.format(n),
             setup=f'lst = list(range({size}))',
             number=100)
      for n in range(0, size, 10)]

plt.plot(ts, 'ro');

### Appending

In [ ]:
ns = np.linspace(100, 10000, 50, dtype=int)

ts = [timeit('lst.append(None)',
             setup=f'lst = list(range({n}))',
             number=1000)
      for n in ns]

plt.plot(ns, ts, 'ro');

### Insertion & Deletion

In [ ]:
ns = np.linspace(100, 10000, 50, dtype=int)

# timing insertion at front
ins_ts = [timeit('lst.insert(0, None)',
                 setup=f'lst = list(range({n}))',
                 number=100)
          for n in ns]

# timing deletion at front
del_ts = [timeit('del lst[0]',
                 setup=f'lst = list(range({n}))',
                 number=100)
          for n in ns]

plt.plot(ns, ins_ts, 'go')
plt.plot(ns, del_ts, 'ro');

### Concatenation

In [ ]:
def concat1(lst1, lst2):
    for x in lst2:
        lst1.append(x)
    return lst1

def concat2(lst1, lst2):
    lst1.extend(lst2)
    return lst1

def concat3(lst1, lst2):
    return lst1 + lst2

In [ ]:
ns = np.linspace(100, 10000, 50, dtype=int)

ts = [[timeit('{}(lst1, lst2)'.format(fn),
              setup=f'lst1 = list(range({n})); lst2 = list(range({n}))',
              number=100,
              globals=globals())
       for n in ns] 
      for fn in ('concat1', 'concat2', 'concat3')]

plt.plot(ns, ts[0], 'ro')
plt.plot(ns, ts[1], 'go')
plt.plot(ns, ts[2], 'bo');

### Conclusion

While index-based updates and appends are $O(1)$, operations that may require changing the relative positions of elements en masse (e.g., insertion, deletion, concatenation) are $O(N)$. 

This is due to the **monolithic, contiguous memory allocation** of the array used to hold references to elements in the list.

## 2. Wishlist

For applications that frequently perform operations such as insertions & deletions (which are inefficient in an array-backed list) we would like to have an improved storage mechanism that:

- does **not** require monolithic, contiguous memory allocation,
- supports flexible and efficient reorganization of substructures (e.g., for concatenation and relocation),
- and allows us to build other commonly needed operations (e.g., for indexing, iteration, search) on top of it.

## 3. Mechanisms

### 3.1. Two-Element Lists

In [ ]:
# data items
i1 = 'lions'
i2 = 'tigers'
i3 = 'bears'
i4 = 'oh, my'

In [ ]:
# creating individual "links"
l1 = [i1, None]
l2 = [i2, None]
l3 = [i3, None]
l4 = [i4, None]

In [ ]:
# link-ing them together
l1[1] = l2
l2[1] = l3
l3[1] = l4

In [ ]:
# data access
head = l1

In [ ]:
head[1][1][1][0]

In [ ]:
# iteration
def link_iterator(l):
    while l:
        yield l[0]
        l = l[1]

In [ ]:
for x in link_iterator(head):
    print(x)

In [ ]:
# prepending
i0 = 'walruses'

In [ ]:
l0 = [i0, head]

In [ ]:
head = l0

In [ ]:
for x in link_iterator(head):
    print(x)

In [ ]:
# insertion
i2_5 = 'elephants'

In [ ]:
l2_5 = [i2_5, l3]
l2[1] = l2_5

In [ ]:
for x in link_iterator(head):
    print(x)

In [ ]:
# deletion
head = head[1]

In [ ]:
for x in link_iterator(head):
    print(x)

### 3.2. "Link" objects

In [ ]:
class Link:
    def __init__(self, val, next=None):
        self.val = val
        self.next = next

In [ ]:
# manually constructing a list
l1 = Link(i1)
l2 = Link(i2)
l3 = Link(i3)
l4 = Link(i4)

l1.next = l2
l2.next = l3
l3.next = l4

In [ ]:
# data access
head = l1
head.next.next.next.val

In [ ]:
# iteration
def link_iterator(l):
    while l:
        yield l.val
        l = l.next

In [ ]:
for x in link_iterator(head):
    print(x)

## 4. Linked Data Structures

### 4.1 Linked List

In [ ]:
class LinkedList:
    class Link:
        def __init__(self, val, next=None):
            self.val = val
            self.next = next

    def __init__(self):
        self.head = None
        
    def prepend(self, val):
        self.head = LinkedList.Link(val, self.head)
        
    def __iter__(self):
        l = self.head
        while l:
            yield l.val
            l = l.next
            
    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [ ]:
l = LinkedList()
for x in range(10):
    l.prepend(x)
l

### 4.2 Binary Tree

In [ ]:
class BinaryLink:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [ ]:
# an "expression tree" representing the arithmetic expression ((5+3)*(8-4))
t = BinaryLink('*')
t.left = BinaryLink('+')
t.left.left  = BinaryLink('5')
t.left.right = BinaryLink('3')
t.right = BinaryLink('-')
t.right.left  = BinaryLink('8')
t.right.right = BinaryLink('4')

In [ ]:
def print_expr_tree(t):
    if t:
        if not t.val.isdigit():
            print('(', end='')
        print_expr_tree(t.left)
        print(t.val, end='')
        print_expr_tree(t.right)
        if not t.val.isdigit():
            print(')', end='')

In [ ]:
print_expr_tree(t)

### 4.3 N-ary Tree

In [ ]:
class NaryLink:
    def __init__(self, val, n=2):
        self.val = val
        self.children = [None] * n
        
    def __getitem__(self, idx):
        return self.children[idx]
    
    def __setitem__(self, idx, val):
        self.children[idx] = val
        
    def __iter__(self):
        for c in self.children:
            yield c

In [ ]:
root = NaryLink('Kingdoms', n=5)

root[0] = NaryLink('Animalia', n=35)
root[1] = NaryLink('Plantae', n=12)
root[2] = NaryLink('Fungi', n=7)
root[3] = NaryLink('Protista', n=5)
root[4] = NaryLink('Monera', n=5)

root[2][0] = NaryLink('Chytridiomycota')
root[2][1] = NaryLink('Blastocladiomycota')
root[2][2] = NaryLink('Glomeromycota')
root[2][3] = NaryLink('Ascomycota')
root[2][4] = NaryLink('Basidiomycota')
root[2][5] = NaryLink('Microsporidia')
root[2][6] = NaryLink('Neocallimastigomycota')

def tree_iter(root):
    if root:
        yield root.val
        for c in root:
            yield from tree_iter(c)

In [ ]:
for x in tree_iter(root):
    print(x)